0x56c145509b9b5d613236ffd6810c037130b20878243e99728772716a89dc70f6
- Business: User Expansion Strategy -> False: Info/Announcement


$cake price
- token reward adjustment
- ???? https://snapshot.org/#/cakevote.eth/proposal/0x8d22df8165ffda034dba6a092ee9743e769ca466813b4b162e3646a62e4a1c10


0xb0b9c229eab715c2752b3cdb652600d1e4fdef21d227de8e037ab9f50e290be5
- Business Fee? trading liquidity?

crypto wiki 긁어서 해당 단어의 설명글을 바탕으로 그 단어의 어조 파악?

In [458]:
import pandas as pd

class_cols = ["Class A", "Class B", "Class C"]

df = pd.read_excel("proposals_preprocess_0812_cleaned.xlsx")
df = df.replace('', None)
df.shape, df[~df[class_cols].isna().all(axis=1)].shape

((20750, 39), (264, 39))

In [459]:
temp = pd.read_excel("proposals_preprocess_0812_cleaned_cached.xlsx")
temp = temp.replace('', None)
temp = temp[~temp[class_cols].isna().all(axis=1)]
temp.shape

(1195, 39)

In [460]:
for d in [df, temp]:
    d["Class A"] = d["Class A"].str.strip()
    d["Class B"] = d["Class B"].str.strip()
    d["Class C"] = d["Class C"].str.strip().str.lower()

    for k, v in {
        "User expanshion Strategy": "User Expansion Strategy",
        "User expansion Strategy": "User Expansion Strategy",
        "User expansion": "User Expansion Strategy",
        "token reward adjustment": "Token Reward Adjustment",
        "token allocation": "Token Allocation",
        "Add new token": "Expansion",
        "Reward adjustment": "Reward Adjustment",
        "Liquidity management": "Liquidity Management",
        "inclusive": "Inclusive",
        "restrictive": "Restrictive",
        "-": None
    }.items():
        d.loc[d["Class B"] == k, "Class B"] = v

for row in temp.iterrows():
    for c in class_cols:
        df.loc[df["id"] == row[1].id, c] = row[1][c]

df.shape, df[~df[class_cols].isna().all(axis=1)].shape


((20750, 39), (1196, 39))

In [461]:
df[["Class A", "Class B"]].value_counts(dropna=False).sort_index() #.to_csv("class_counts.csv")

Class A           Class B                
-                 NaN                            1
Business          Expansion                     41
                  Fee                           59
                  New Business Proposal          5
                  Rate                           4
                  Reward Adjustment             33
                  User Expansion Strategy       12
F1                request money                269
F2                no detailed content            6
F3                Off-topic                     37
F4                Info/Announcement             64
Governance        Inclusive                     21
                  Restrictive                    1
                  Security                       1
                  adjust                         1
                  feature improvement            5
                  NaN                            5
Opearations       Team                           2
Operations        Feature Improvement   

In [462]:
groups = df.groupby(["Class A", "Class B", "Class C"], dropna=False)
temp = []
for name, group in groups:
    temp.append(group.head(10))
temp = pd.concat(temp)
temp[["id", "link", "title", "body", "author", "votes", "choices", "space_id", "Class A", "Class B", "Class C"]].to_excel("sample_category.xlsx", engine="openpyxl", index=False)

# NLP Cleansing

Remove stopwords

In [463]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)
nltk.download('wordnet', quiet=True)
stop_words = set(stopwords.words('english'))

In [464]:
# Remain only lower case english words
for v in ["title", "body"]:
    df[f"{v}_nlp"] = df[f"{v}_lower"].astype(str) \
        .str.replace("[^a-z\n]", " ", regex=True).str.strip() \
            .apply(lambda x: ' '.join(
            (w for w in word_tokenize(x) if not w in stop_words)))

### Remove Words

In [383]:
!pip install -q --user nltk pattern xlsxwriter


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [465]:
from pattern.en import lemma, singularize
from nltk.corpus import wordnet
import numpy as np

skip_words = lambda arr: np.vectorize(lambda x: ' '.join(
        xx for xx in x.split() if xx not in arr
    ).strip())
    
is_word_in_dictionary = lambda word: any(
    w for w in set([word, singularize(word), lemma(word)]) 
    if len(wordnet.synsets(w)) > 0
)
    
print(is_word_in_dictionary("writing"))  # True (-ing로 끝나는 동사가 기본형으로 변환됨)
print(is_word_in_dictionary("apples"))  # True (복수형 단어가 기본형으로 변환됨)
print(is_word_in_dictionary("ate"))     # True (과거형으로 변환된 단어가 기본형으로 변환됨)
print(is_word_in_dictionary("qwerty"))  # False (사전에 없는 단어)

True
True
True
False


Remove protocol names and coin symbol

In [466]:
import re
import requests

words = []

# dao symbol
func = np.vectorize(lambda x: re.sub(r"\.eth", "", x).strip())
words.append(
    func(df["space_id"].unique())
)

# dao symbol except "gov", "dao", "vote"
func = np.vectorize(lambda x: re.sub("\.|\-", " ", re.sub(r"\.eth|gov|dao|vote", "", x)).strip())
words.append(
    np.sort(np.array([word for sentence in func(df["space_id"].unique()) for word in sentence.split() if len(word) > 1]))
)

# All of coin symbol
response = requests.get("https://api.coingecko.com/api/v3/coins/list")
words.append(
    np.array([x for x in set(coin['symbol'].lower() for coin in response.json())])
)
words.append(
    np.array([x for x in set(coin['symbol'].lower()+"s" for coin in response.json())])
)
words.append(
    np.array([x for x in set(coin['name'].lower() for coin in response.json())])
)

# Communication Channel Name
words.append(
    np.array(["tiktok", "linkedin", "github", "gitbook", "facebook", "reddit", "discord"])
    # medium...
)

print([v.shape for v in words])
words = np.unique(np.concatenate(words))
words.shape

[(245,), (264,), (11429,), (11429,), (14354,), (7,)]


(34572,)

In [467]:
func = skip_words(words)
df["body_nlp"] = func(df["body_nlp"].values)

In [469]:
word_counts = Counter((
    item 
    for sub in (
        str(x).split() for x in df["body_nlp"]
    ) 
    for item in sub
))
print(len(word_counts))


words = [x for x in word_counts.keys() 
    if (("dao" in x and len(x) > 4) or ("swap" in x and len(x) > 5) or "frax" in x)
    and not is_word_in_dictionary(x)]
print(len(words))

func = skip_words(words)
df["body_nlp"] = func(df["body_nlp"].values)

50670
0


Remove rare words

In [470]:
import numpy as np
from collections import Counter

word_counts = Counter((
    item 
    for sub in (
        str(x).split() for x in df["body_nlp"]
    ) 
    for item in sub
))
len(word_counts)

single = set([
    word 
    for word, count in word_counts.items() 
    if  count <= 100 and not is_word_in_dictionary(word)
])
print(len(single))
single

26674


{'crscia',
 'dbochman',
 'fwgs',
 'nueral',
 'emilianos',
 'yveth',
 'basile',
 'landshares',
 'rockaway',
 'zal',
 'memeability',
 'mimicfi',
 'qimp',
 'rpsbet',
 'mikr',
 'rewardgauge',
 'govuk',
 'nbn',
 'txbrowser',
 'crowdsourced',
 'microgravity',
 'problemas',
 'anymagik',
 'microservices',
 'yng',
 'blockdaemon',
 'cornelius',
 'newscience',
 'higherhigh',
 'crypo',
 'angelvault',
 'extempt',
 'berea',
 'cryptocomic',
 'gascostbean',
 'terralabs',
 'kasa',
 'escan',
 'consideation',
 'siriwat',
 'foveated',
 'qmcvmeu',
 'hu',
 'tickbitmap',
 'minigames',
 'parallelizationand',
 'beaconcha',
 'islandid',
 'kycs',
 'usership',
 'sophie',
 'jewfuckerr',
 'bamm',
 'exomads',
 'rerouting',
 'npos',
 'publius',
 'listingmanager',
 'sveltekit',
 'vestingvault',
 'triaged',
 'bicocca',
 'upsells',
 'vuhusx',
 'wetrust',
 'imageuploader',
 'pcsv',
 'congruentlabs',
 'gset',
 'roshangar',
 'mirrorx',
 'jhan',
 'shopee',
 'mahoro',
 'conjunto',
 'nene',
 'teamup',
 'meatspace',
 'tobas',


In [471]:
func = skip_words(single)
df["body_nlp"] = func(df["body_nlp"].values)

Remove remain crypto terms

In [472]:
word_counts = Counter((
    item 
    for sub in (
        str(x).split() for x in df["body_nlp"]
    ) 
    for item in sub
))
len(word_counts)

temp = pd.DataFrame({"words": word_counts.keys()})
temp["count"] = temp["words"].apply(lambda x: word_counts[x])
temp["dict"] = temp["words"].apply(lambda x: is_word_in_dictionary(x))
temp[temp["dict"]==False].sort_values(by="count").to_csv("word_counts.csv", index=False)

In [480]:
words = [x for x in word_counts.keys() if 
    any(v in x for v in (
        # "kyc", "llama", "coinbase",  "codebase", "dune", "coingecko", "metamask", "certik", "okx", "defillama"
        #  "unity", "opensea", "etherscan", 
        "aave", "cake", "decentraland", "arfc", "kalmy",
        "bayc", "zk", "beethoven", "tmdc", "immunefi", "karpatkey", "alchem",
        "fest", "vefxs", "osnap", "pixelcraft", "botb", "filipv",  "mayc",
        "bancor", "poap", "stargate", "apechain", "karpatkey", "arrakis",
        "gotchi", "openzeppelin", "agd", "pvfd", "merkl", "apecoins",
        "certik", "ethdenver", "indexcoop", "dopewars", "jbp", "gamefi",
        "halborn", "bgd", "baller", "wintermute", "dsr", "stax", "mario", "jpegd", "quo", "fip",
        "zora", "euroc", "kh",

        # "velodrome", "xdai", "jarvis", "circle", "ichi", "polkadot", "galxe", "aip",
        #  "parcel", "gauntlet","convex", "fantom", "tokenlogic", "gyroscope", "sushi",
        # "beanstalk",  "fox","chainlink", "thorchain", "cayman", "layerzero", "moon", 
        
        "krause", "aura", "crv", "vlcvx", "ageur", "vebal", "vleqb", "vesdt", "veextra", "veangle", "vlmgp" "kpis", "matic", "aggold",
        )) and
        not is_word_in_dictionary(x) ]
len(words)

63

In [481]:
words

['dsr',
 'arfc',
 'bayc',
 'mayc',
 'certik',
 'vesdt',
 'krause',
 'mario',
 'stargate',
 'bancor',
 'vecake',
 'poap',
 'gotchiid',
 'botb',
 'fest',
 'zora',
 'programmatically',
 'kh',
 'pixelcraft',
 'filipv',
 'jbp',
 'osnap',
 'beethovenx',
 'openzeppelin',
 'ethdenver',
 'wmatic',
 'vecrv',
 'ageur',
 'bgd',
 'vlaura',
 'karpatkey',
 'aavev',
 'immunefi',
 'stkaave',
 'wintermute',
 'merkl',
 'halborn',
 'agd',
 'arrakis',
 'gotchiverse',
 'alchemica',
 'quo',
 'gamefi',
 'vebal',
 'vlcvx',
 'vefxs',
 'ballers',
 'euroc',
 'indexcoop',
 'stax',
 'vleqb',
 'veangle',
 'decentralands',
 'veextra',
 'fip',
 'tmdc',
 'pvfd',
 'jpegd',
 'aggold',
 'apecoins',
 'apechain',
 'dopewars',
 'kalmy']

In [482]:
func = skip_words(words)
df["body_nlp"] = func(df["body_nlp"].values)

Lemmatization: extract original word

In [483]:
func = lambda x: ' '.join(
       (singularize(xxx) for xxx in (lemma(xx) for xx in x.split()))
    ).strip()
func = np.vectorize(func)
df["body_nlp"] = func(df["body_nlp"].values)

In [494]:
df.loc[(df["body_nlp"]=="nan") | (df["body_nlp"]=="undefine")  , "body_nlp"] = None

In [495]:
df = df.replace('', None)
print(df[df["body_nlp"].isna()]["space_id"].value_counts())
df[df["body_nlp"].isna()][["title", "body", "space_id"]]

index-coop.eth            91
conic-dao.eth             61
cvx.eth                   55
leagueoflils.eth          54
gauges.aurafinance.eth    40
muuu.eth                  28
aurafinance.eth           15
thelanddaonft.eth          1
Name: space_id, dtype: int64


,title,body,space_id
387,351: Compensation for Anniversary Art Inclusio...,NaN,leagueoflils.eth
388,352: Buy an Alien CryptoPunk using stETH,NaN,leagueoflils.eth
391,"354: Nouns DAO V3 upgrade, including Nouns For...",NaN,leagueoflils.eth
393,355: Nounify Ethereum Singapore with Public Goods,NaN,leagueoflils.eth
395,"369: Commercialization of the ""Noggle Changer""!-",NaN,leagueoflils.eth
...,...,...,...
19836,USDT LAV for week of 20th Apr 2023,NaN,conic-dao.eth
19837,USDT LAV for week of 25th Apr 2024,NaN,conic-dao.eth
19838,USDT LAV for week of 29th June 2023,NaN,conic-dao.eth
19839,USDT LAV for week of 4th May 2023,NaN,conic-dao.eth


Result of Cleansing

In [500]:
temp = df[~df["body_nlp"].isna()].copy()
temp["nlp_len"] = temp["body_nlp"].apply(len)
temp[temp["nlp_len"] < 30][["body_nlp"]].value_counts()

body_nlp                     
proposal vote newest proposal    12
background reason                12
vote follow proposal              6
allocate vote proposal            4
holder vote lock product          4
holder vote k lock product        4
prolong incentive have day        2
proposal vote latest proposal     2
proposal vote proposal            2
distribute utc price              2
amplification quorum              2
treasury info vote vote           1
read com                          1
vote support sinful committee     1
vote support committee            1
propose decrease staker           1
propose multisig repay pr         1
vote proposal                     1
vote committee                    1
quorum acquisition                1
return recoverable take forum     1
tldr noun proposal                1
see context option grant          1
see forum                         1
start saturday right week         1
state leave do                    1
user                              

In [497]:
temp = temp[temp["nlp_len"] < 100]["space_id"].value_counts()
print(temp.shape, temp.sum())
temp[temp>10]

(149,) 1980


cakevote.eth                          379
index-coop.eth                        343
snapshot.dcl.eth                      262
cvx.eth                               115
unidexapp.eth                          42
decentralgames.eth                     30
rndrnetwork.eth                        25
glmrapes.eth                           25
futera.eth                             24
abracadabrabymerlinthemagician.eth     23
badgerdao.eth                          23
orbapp.eth                             21
derify.eth                             20
prismafinance.eth                      19
arbitrumfoundation.eth                 18
gnosis.eth                             18
timelessfi.eth                         18
stakewise.eth                          18
theheaddao.eth                         17
gramdao.eth                            17
morpho.eth                             16
snxgov.eth                             16
lido-snapshot.eth                      16
dinero.xyz                        

# With Categorized dataset

In [501]:
df_ = df[~(df["body_nlp"].isna())].copy()
df_.shape

(20405, 41)

In [502]:
df_class = df_[df_[class_cols].notnull().any(axis=1)].reset_index(drop=True).sort_values(by=['body_nlp'])
df_class["nlp_len"] = df_class["body_nlp"].apply(len)
df_class[df_class["nlp_len"] < 60][["body_nlp"]].value_counts().sort_values()

body_nlp                                                  
propose increase feed increase leverage favmy decision         1
amount receive receive receive                                 1
android invalid question option                                1
avatar yeah invalid question option                            1
begin skyrocket amaze meeting acceleration skyrocket           1
                                                              ..
official airdrop limit offer guarantee                         3
eagerly anticipate distribution enthusiastic participation     3
vote decide whether technical review network foundation        7
official announcement airdrop distribute supply                8
holder choose change week control want reward                 26
Length: 76, dtype: int64

In [503]:
df_not = df_[df_[class_cols].isnull().all(axis=1)].reset_index(drop=True).sort_values(by=['body_nlp'])

df_class.shape, df_not.shape

((1160, 42), (19245, 41))

In [504]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(df_class["body_nlp"])
# cos_mat = cosine_similarity(X, X)
# print(cos_mat.shape)


# threshold = 0.8
# selected_rows = []
# visited = set()

# for i in range(len(df_class)):
#     if i not in visited:
#         selected_rows.append(i)
#         for j in range(i + 1, len(df_class)):
#             if cos_mat[i][j] >= threshold:
#                 visited.add(j)

# selected_rows = np.unique(np.array(selected_rows))
# df_class = df_class.loc[selected_rows]
# df_class.shape

In [505]:
df_body = pd.concat([df_class["body_nlp"], df_not["body_nlp"]])

X = vectorizer.fit_transform(df_body)

df_class_body = X[:df_class.shape[0]]
df_not_body = X[df_class.shape[0]:]

cos_mat = cosine_similarity(df_class_body, df_not_body)
print(cos_mat.shape)

(1160, 19245)


In [506]:
columns = ["similarity", "id_c", "space_id_c", "title_c", "body_c", "link_c", "id_n", "space_id_n", "title_n", "body_n", "link_n"]
df_columns = ["id", "space_id", "title", "body", "link"]

def print_line(data):
    print("[ID]:", data["id"])
    print("[Space ID]:", data["space_id"])
    print("[Title]:", data["title"])
    print(data["body"])

def find_similar(threshold: int, threshold_upper: int=2, doPrint: bool = False):
    global cos_mat, df_class, df_not, columns, class_cols, df_columns

    temp = {k: None for k in columns + class_cols}
    result = []
    over = (cos_mat >= threshold) & (threshold_upper > cos_mat)

    for i in (i for i, v in enumerate(over.sum(axis=1)) if v > 0):
        y = df_class.iloc[i]
        if doPrint:
            print("-------------------------------------")
            print("-------------------------------------")
            print("[Category]:", y["Class A"], y["Class B"], y["Class C"], sep="\t")
            print_line(y)
            print("-------------------------------------")
        for v in df_columns:
            temp[v+"_c"] = y[v]
        for v in class_cols:
            temp[v] = y[v]

        
        for j in (idx for idx, v in enumerate(over[i]) if v): 
            x = df_not.iloc[j]
            if doPrint:
                print(f"{cos_mat[i, j]}")
                print_line(x)
                print("======================\n\n")
            temp["similarity"] = cos_mat[i, j]
            for v in df_columns:
                temp[v+"_n"] = x[v]
            result.append(temp.copy())
    return pd.DataFrame(result)

In [507]:
panel = [2, 1, 0.9, 0.8, 0.75] # 0.6은 별로
df_panel = {}
for i in range(len(panel)-1):
    result = find_similar(threshold=panel[i+1], threshold_upper=panel[i])
    df_panel[f"over{panel[i+1]}"] = result
    print(result.shape)

(79, 14)
(109, 14)
(163, 14)
(220, 14)


In [508]:
df_panel["over1"][["title_c", "body_c", "title_n", "body_n"]].head()

,title_c,body_c,title_n,body_n
0,Season 4 Governance Council Election,Abstract\nAs the CUIP-004 - Governance Council...,Season 3 Governance Council Election,Abstract\nAs the CUIP-004 - Governance Council...
1,Season 2 Governance Council Election,Abstract\nAs the CUIP-004 - Governance Council...,Season 3 Governance Council Election,Abstract\nAs the CUIP-004 - Governance Council...
2,Fantastic news for all Angle users!,Calling all Angle users! We're delighted to sh...,Absolutely thrilling news for all Lido users!,Calling all Lido users! We're delighted to sha...
3,Fantastic news for all rhinofi users!,Calling all rhinofi users! We're delighted to ...,Absolutely thrilling news for all Lido users!,Calling all Lido users! We're delighted to sha...
4,Fantastic news for all Angle Protocol users!,Calling all Angle Protocol users! We're deligh...,Absolutely thrilling news for all Lido users!,Calling all Lido users! We're delighted to sha...


In [509]:
output_file = 'similar_categorized_filtered.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for k in df_panel.keys():
        df_panel[k].to_excel(writer, sheet_name=k, index=False)

In [510]:
df = df.sort_values(by=["id"])

pool = set()
for k in df_panel.keys():
    if df_panel[k].empty:
        continue
    df_panel[k] = df_panel[k].sort_values(by=["id_n"])

    groups = df_panel[k].groupby(["id_n", "Class A", "Class B"], dropna=False)
    t = groups.count().index.to_frame().reset_index(drop=True)
    temp = t["id_n"].value_counts()
    temp_x = temp[temp>1].index
    temp = temp[~(temp>1)].index
    print(k,  t[t["id_n"].isin(temp_x)])
    temp = t[t["id_n"].isin(temp) & ~t["id_n"].isin(pool)]

    if (df.loc[df["id"].isin(temp["id_n"])]["id"].values == temp["id_n"].values).all():
        df.loc[df["id"].isin(temp["id_n"]), ["id", "Class A", "Class B"]] = temp.values
        print(temp.values.shape)
        pool.update(set(temp.values.flatten()))

over1 Empty DataFrame
Columns: [id_n, Class A, Class B]
Index: []
(19, 3)
over0.9 Empty DataFrame
Columns: [id_n, Class A, Class B]
Index: []
(31, 3)
over0.8 Empty DataFrame
Columns: [id_n, Class A, Class B]
Index: []
(57, 3)
over0.75 Empty DataFrame
Columns: [id_n, Class A, Class B]
Index: []
(58, 3)


In [511]:
df[df[class_cols].notnull().any(axis=1)].shape

(1359, 41)

In [512]:
df.to_excel("proposals_preprocess_0812_nlp.xlsx", index=False)

# Without Categorized Dataset 

In [51]:
!pip install -q networkx


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [513]:
df_not = df_not[~df_not["id"].isin(df_panel["over1"]["id_n"])]
df_not = df_not[~df_not["id"].isin(df_panel["over0.9"]["id_n"])]

In [514]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_not["body_nlp"])
cos_mat = cosine_similarity(X, X)
cos_mat.shape

(19195, 19195)

In [515]:
import networkx as nx

def find_similar_cluster(threshold: int, threshold_upper:int=2):
    G = nx.Graph()
    for idx in range(len(df_not)):
        G.add_node(df_not.iloc[idx]["id"])

    over = (cos_mat >= threshold) & (threshold_upper > cos_mat) 
    for i in range(len(df_not)):
        for j in range(i + 1, len(df_not)):
            if over[i, j]:
                G.add_edge(df_not.iloc[i]["id"], df_not.iloc[j]["id"], weight=cos_mat[i, j])

    filtered_components = [
        df_not[df_not["id"].isin(list(c))][["id", "space_id", "title", "body", "link", "Class A", "Class B", "Class C"]]
        for c in list(nx.connected_components(G)) 
        if len(c) >= 2
    ]
    filtered_components = sorted(filtered_components, key=lambda x: len(x), reverse=True)

    rows = 0
    output_file = f'df_not_{threshold}_nlp.xlsx'
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        for i, component in enumerate(filtered_components):
            component.to_excel(writer, sheet_name=str(i), index=False)
            rows += component.shape[0]
    print(f"Number of rows with threshold {threshold}: {rows}")
    return filtered_components

In [516]:
panel = [2, 1, 0.9, 0.8]
for i in range(len(panel)-1):
    filtered_components = find_similar_cluster(threshold=panel[i+1], threshold_upper=panel[i])
    print(panel[i+1], len(filtered_components))

Number of rows with threshold 1: 1915
1 579
Number of rows with threshold 0.9: 3397
0.9 980
Number of rows with threshold 0.8: 2686
0.8 570
